In [1]:
# transformers not support NumPy 2.0 yet
!pip install -q numpy~=1.26.4 transformers~=4.46.2
!pip install -q datasets~=3.2.0 pydantic~=2.10.4
!pip install -q seqeval~=1.2.2 evaluate~=0.4.3

# 訓練 PII 偵測模型

在這個筆記本中，我們將展示如何使用 `transformers` 套件訓練 PII (個人識別資訊) 偵測模型。我們將使用 `transformers` 套件中的 [`Trainer`](https://huggingface.co/docs/transformers/main_classes/trainer) 類別來微調一個 Encoder-Only 架構的 BERT 模型。我們將利用到標記分類 ([Token classification](https://huggingface.co/docs/transformers/tasks/token_classification)) 進行下游任務 (downstream task) 的訓練。

標記分類為句子中的單詞分配標籤 (Label)。最常見的標記分類任務之一是命名實體識別 (NER)。NER 試圖為句子中的每個實體找到一個標籤，例如人名、地點或組織。

In [2]:
import pandas as pd
import numpy as np
import evaluate

from transformers import (
  AutoTokenizer,
  AutoModelForTokenClassification,
  DataCollatorForTokenClassification,
  pipeline,
  TrainingArguments,
  Trainer,
)
from datasets import load_dataset, DatasetDict

from pydantic import BaseModel
from pprint import pprint

import ast
import torch

# 檢查是否有 GPU 可以使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)

## 下載資料

從 Kaggle 下載 PII External Dataset，並解壓縮到 `sample_data` 資料夾。

In [3]:
# 從 Kaggle 下載 PII External Dataset
!curl -L -o ./sample_data/pii-external-dataset.zip \
  https://www.kaggle.com/api/v1/datasets/download/alejopaullier/pii-external-dataset

# 解壓縮
!unzip -o -q ./sample_data/pii-external-dataset.zip -d ./sample_data/

解壓縮後的資料為 CSV 檔案，`load_dataset()` 函數可以讀取 CSV 檔案。

無論數據集存儲在哪裡，[Datasets](https://huggingface.co/docs/datasets/loading) 都可以幫助您加載它。數據可以存儲在 Hugging Face Hub、本地機器上、在 Github 中，以及在內存數據結構中，也可以在 Pandas DataFrame 之間轉換。在接續的實戰中，我們也都會用到 `load_dataset()` 函數加載數據集。

In [4]:
# The full `train` split
immutable_dataset = load_dataset('csv', data_files='sample_data/pii_dataset.csv', split='train')

### 資料包含什麼？

In [5]:
# 顯示原始資料中包含的 features 以及筆數
immutable_dataset

In [6]:
# 檢視資料集中的第一筆資料
pd.set_option('display.max_colwidth', None)
df = pd.DataFrame(immutable_dataset[:1])
# 將 'text', 'tokens', labels' 三個欄位的順序移至前面
df.insert(0, 'labels', df.pop('labels'))
df.insert(0, 'tokens', df.pop('tokens'))
df.insert(0, 'text', df.pop('text'))
df

這個表格結構，包含多項資訊，在這個筆記本中，我們將使用：

* `text`: 一個完整字符串，包含 PII 資訊，將用作輸入。

* `token`: 依據 `text` 切割後的字詞及字符，將用作訓練。

* `label`: 對應每個 `token` 的標記，其中包含 PII 資訊的類別，將用作訓練。

### 資料前處理

保留必要 features: `text`, `tokens` 及 `labels`。

In [7]:
# 保留必要 features: 'text', 'tokens', 'labels'
dataset = immutable_dataset.remove_columns([
  'document', 'trailing_whitespace', 'prompt', 'prompt_id', 'name',
  'email', 'phone', 'job', 'address', 'username', 'url', 'hobby', 'len'])

從 CSV 檔案讀取後的 `tokens` 與 `labels` 是字符串型別，我們需要將其轉換為 list 型別。

In [8]:
type(dataset[0]['tokens'])

In [9]:
type(dataset[0]['labels'])

以下程式碼使用 `dataset.map` 方法來轉換數據集中的每個元素。具體來說，它將每個元素中的 `tokens` 和 `labels` 字段從字符串格式轉換為其對應的 Python 數據結構：

* `dataset.map`: map 方法用於對數據集中的每個元素應用一個函數，並返回一個新的數據集。這個方法通常用於數據預處理和轉換。

* `lambda x: { ... }`: 這是一個匿名函數（lambda 函數），它接受一個輸入 x，並返回一個字典。x 代表數據集中的一個元素。

* `ast.literal_eval`: `ast.literal_eval` 是 Python 的 ast 模塊中的一個函數，它可以將字符串轉換為對應的 Python 數據結構，如列表、字典等。

In [10]:
# 將 'tokens' 與 'labels' 從 string 轉換為 list (CSV 檔案讀取後會變成 string)
dataset = dataset.map(
  lambda x: {
    'tokens': ast.literal_eval(x['tokens']),
    'labels': ast.literal_eval(x['labels'])
  }
)

以上程式碼的作用是對數據集中的每個元素進行轉換，將 `tokens` 和 `labels` 字段從字符串格式轉換為對應的 Python 數據結構。這樣做的目的是便於後續的數據處理和模型訓練。轉換後的數據集將包含已解析的 `tokens` 和 `labels`，而不再是原始的字符串格式。

透過過濾數據集中的元素，確保每個元素中的 `tokens` 和 `labels` 字段的長度相等，確保 `tokens` 與 `labels` 長度相等，避免有缺失的情況。

In [11]:
# 確認 tokens 長度與 labels 長度相等，避免有缺失的情況
dataset = dataset.filter(lambda x: len(x['tokens']) == len(x['labels']))

將 `tokens` 欄位重新命名為 `words` 避免與後面的 tokens 概念混淆。

In [12]:
# 將 tokens 欄位重新命名為 words 避免與後面的 tokens 概念混淆
dataset = dataset.rename_column('tokens', 'words')

# 顯示處理後的資料
dataset

將數據集分割為訓練集、驗證集和測試集。

* 保留 0.1% 的數據作為測試集，為了控制課程演示所以採用 `shuffle=False`，確保在不同運行中訓練集、驗證集和測試集保持一致。您可以試試看 `shuffle=True` 來打亂數據。

* 將剩餘的數據集分割為 80% 的訓練集和 20% 的驗證集。

* 創建包含訓練集、驗證集和測試集的數據集字典。


In [13]:
# Reserve 0.1% of the training set for testing
test_dataset = dataset.train_test_split(
  test_size=0.001, # 0.1% of the data is used for testing
  shuffle=False, # Ensure that train and validation sets are the same across runs
  )
# Split into 80% training and 20% validation sets
train_dataset = test_dataset['train'].train_test_split(
  test_size=0.2, # 20% of the data is used for validation
  shuffle=False, # Ensure that train and test sets are the same across runs
  )
dataset = DatasetDict({
  'train': train_dataset['train'],
  'validation': train_dataset['test'],
  'test': test_dataset['test'],
  })
# 顯示處理後的資料
dataset

In [14]:
# 顯示前 first_n_data 筆資料
first_n_data = 2
pd.set_option('display.max_colwidth', None)
pd.DataFrame(dataset['train'].select(range(first_n_data)))

為了方便我們說明及檢視資料，所以我們期望將 `words` 與 `labels` 並列對齊顯示如下：

| 1      | 2   | 3              | 4              |
|--------|-----|----------------|----------------|
| Hello, | I'm | Nicholas       | Moore,         |
| O      | O   | B-NAME_STUDENT | I-NAME_STUDENT |

於是我們定義一個函數，輸入為單一筆數據及最大列印長度，輸出為對齊後的數據。

In [15]:
def display_words_labels(x, max_display):
    words = x['words'][:max_display]
    labels = x['labels'][:max_display]
    line1 = ""
    line2 = ""
    for word, label in zip(words, labels): # 逐一取出 word 與 label
        # 計算 word 與 label 的最大長度, 並將 word 與 label 用空白補齊至相同長度
        max_length = max(len(word), len(label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += label + " " * (max_length - len(label) + 1)
    pprint(line1, width=200)
    pprint(line2, width=200)

In [16]:
# 並列顯示前 max_display 個 words 與 labels
max_display = 50

# 顯示前 first_n_data 筆資料中的前 max_display 個 words 與 labels
for i in range(first_n_data):
    display_words_labels(dataset['train'][i], max_display)
    print()

### 資料中的 BIO 標注

IOB 格式（Inside, Outside, Beginning 的縮寫），也常被稱為 BIO 格式，是計算語言學中用於標記任務（例如命名實體識別 NER，詞性標記 POS）的常見標記格式。

* B - for the first token of a named entity

* I - for tokens inside named entity's

* O - for tokens outside any named entity

我們需要先了解數據集中包含哪些 BIO 標注，所以我們先從訓練集中取出所有的標籤，並且去除重複值。

In [17]:
# 顯示 BIO 標注
label_names = set()
for data in dataset['train']:
    # 將 labels 欄位轉換為 set 並更新 label_names
    label_names.update(data['labels'])
# convert set to list and sort label names
label_names = list(label_names)
# sort label names
label_names.sort()
print('=== 資料集中的標籤 ===')
pprint(label_names, compact=True)
print()
print(f'標籤數量: {len(label_names)}')

由上，我們可以獲得十個標籤，分別為：

```json
[
  'B-EMAIL', 'B-NAME_STUDENT', 'B-PHONE_NUM', 'B-STREET_ADDRESS', 'B-URL_PERSONAL', 
  'B-USERNAME', 'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'O'
]
```

方便後續訓練過程，我們將建立兩個對照表，一個是將 BIO 標籤映射到一個整數編碼，另一個是將整數編碼映射回 BIO 標籤。

In [18]:
# 整數標籤到 BIO 標籤的映射
id2tag = dict(enumerate(label_names))

print('=== id2tag ===')
pprint(sorted(id2tag.items(), key=lambda x: x[0]), compact=True)

第一張對照表是 0 代表 `B-EMAIL`，1 代表 `B-NAME_STUDENT`，2 代表 `B-PHONE_NUM`，以此類推。

In [19]:
# BIO 標籤到整數標籤的映射
tag2id = dict((v, k) for k, v in id2tag.items())

print('=== tag2id ===')
pprint(sorted(tag2id.items(), key=lambda x: x[1]), compact=True)

反之，建立另一個對照表，將 `B-EMAIL` 映射回 0，將 `B-NAME_STUDENT` 映射回 1，將 `B-PHONE_NUM` 映射回 2，以此類推。

## 訓練參數

當我們在微調或訓練機器學習模型時，有幾個重要的參數需要設定。這些參數會影響模型的訓練效果和性能。以下是這些參數的簡單解釋：

* 批次大小（Batch Size）：每次訓練迭代中使用的樣本數量。假設你有1000個數據點，批次大小為32，這意味著模型每次會使用32個數據點來更新權重。當所有數據點都被使用完一次後，這稱為一個 epoch。較大的批次大小可以加速訓練，但需要更多的內存；較小的批次大小則更穩定，但訓練速度較慢。
* 訓練輪數（Epochs）：完整遍歷訓練數據集的次數。如果你設定 epochs 為10，這意味著模型會完整地看10次訓練數據集。更多的 epochs 可以讓模型學習得更充分，但過多的 epochs 可能會導致過擬合（Overfitting / 模型在訓練數據上表現很好，但在新數據上表現不好）。
* 學習率（Learning Rate）：每次更新模型權重時的步伐大小。學習率決定了模型在每次更新時應該調整多少權重。較高的學習率會使模型快速學習，但可能會跳過最佳解；較低的學習率會使模型穩定學習，但訓練時間較長。學習率需要仔細調整，過高或過低都會影響模型的性能。
* 隨機失活（Dropout）：在訓練過程中隨機忽略一些神經元的比例。Dropout 是一種正則化技術，用於防止過擬合。它通過在每次訓練步驟中隨機忽略一些神經元來強制模型學習更穩健的特徵。適當的 Dropout 可以提高模型的泛化能力，但過高的 Dropout 可能會導致模型欠擬合（Underfitting / 模型在訓練數據和新數據上都表現不好）。
* 權重衰減（Weight Decay）：在每次更新權重時，對權重施加的正則化項。Weight Decay 是另一種正則化技術，用於防止過擬合。它通過在每次更新時對權重施加懲罰，使權重保持較小的值。適當的 Weight Decay 可以提高模型的泛化能力，但過高的 Weight Decay 可能會導致模型欠擬合。

這些參數在模型訓練中扮演著重要角色，影響著模型的學習速度、穩定性和泛化能力。調整這些參數需要根據具體的數據集和任務進行實驗和調整，以找到最佳的組合。

### 訓練設定

In [20]:
# 訓練相關設定
class Config(BaseModel):
  model_name: str = 'dslim/distilbert-NER' # 使用蒸餾模型，降低參數量，加快訓練速度
  saved_model_path: str = 'sample_data/saved_decoder_model' # path to save the trained model
  train_batch_size: int = 4 # size of the input batch in training
  eval_batch_size: int = 4 # size of the input batch in evaluation
  epochs: int = 1 # number of times to iterate over the entire training dataset
  lr: float = 2e-5 # learning rate, controls how fast or slow the model learns
  weight_decay: float = 0.01 # weight decay, helps the model stay simple and avoid overfitting by penalizing large weights.
  tags: list # BIO 標註的標籤列表
  id2tag: dict # 整數標籤到 BIO 標籤的映射
  tag2id: dict # BIO 標籤到整數標籤的映射
  num_tags: int # 標籤數量

config = Config(
  tags=label_names,
  id2tag=id2tag,
  tag2id=tag2id,
  num_tags=len(label_names)
)

## Fine-tuning 前的表現

### 載入預訓練分詞器 (Tokenizer)

In [21]:
# 透過預訓練模型取得 Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
  config.model_name,
)
pprint(tokenizer)

確認為 fast tokenizer，方便後續做資料預處理。

> [Fast tokenizers’ special powers](https://huggingface.co/learn/nlp-course/chapter6/3?fw=pt)

In [22]:
# 確認 tokenizer 是否為 fast tokenizer
tokenizer.is_fast

### 載入預訓練模型

透過 `AutoModelForTokenClassification` 載入預訓練模型，並確認模型的分類數。

In [23]:
model = AutoModelForTokenClassification.from_pretrained(
  config.model_name,
).to(device)

In [24]:
pprint(model)

這是一個典型的 Encoder-Only 架構，其中包含一個 BERT 模型和一個線性分類器。BERT 模型用於提取特徵，線性分類器用於將特徵映射到分類標籤。

觀察預訓練模型的分類數為 9。

```json
(classifier): Linear(in_features=768, out_features=9, bias=True)
```

### Fine-tuning 前的表現

#### 單筆演示分類

In [25]:
# 載入預訓練模型，宣告分類器
classifier = pipeline(
  task="token-classification",
  model=model,
  tokenizer=tokenizer,
  device=device,
)

In [26]:
# 透過預訓練模型預測分類
predict = classifier(dataset['test'][0]['text'])

In [27]:
pprint(predict)

我們將獲得一個句子中被標記的字詞，其中 `word` 為字詞，`entity` 為標籤分類。

```json
[
  {
    'entity': 'B-PER', 
    'word': 'Nicholas'
  }, {
    'entity': 'I-PER',
    'word': 'Moore'
  },
  ...
]
```

#### 批次演示分類

初步了解如何利用 `pipeline` 進行分類，我們將定義一個 `display_words_entity` 函數，輸入為一個句子，輸出為對齊後的字詞及標籤。


In [61]:
def display_words_entity(text, classifier):
    result = classifier(text) # 預測結果
    line1 = ""
    line2 = ""
    for r in result:
        # 取出預測結果中的 word 與 entity
        word = r['word']
        label = r['entity']
        # 計算 word 與 entity 的最大長度, 並將 word 與 entity 用空白補
        max_length = max(len(word), len(label))
        line1 += word + " " * (max_length - len(word) + 1)
        line2 += label + " " * (max_length - len(label) + 1)
    pprint(line1)
    pprint(line2)

In [29]:
# 顯示預訓練模型預測結果，僅顯示有被標注的部分
for val in dataset['test']: # 逐一取出測試資料
  print(f'輸入: {val["text"]}')
  display_words_entity(val["text"], classifier)
  print()

可以留意到，預訓練模型已經具備一定的分類能力，但是由於我們的任務是 PII 偵測，所以我們需要進行 Fine-tuning 來提高模型的性能。同時，我們希望調整分類數，以符合我們的任務需求。

## 訓練模型

### 了解分詞器 (Tokenizer) 行為

首先，我們先試試看透過分詞器將一個句子轉換為 tokens。

In [30]:
input_labels = dataset['train'][0]["labels"]
input_words = dataset['train'][0]["words"]
# 輸入為將 'text' 切割後的字詞及字符
pprint(input_words, compact=True)

由於輸入已經預先分詞 (e.g., split into words)，所以我們將 `is_split_into_words` 設為 `True`。

In [31]:
# 對 input_words 進行分詞
input_tokenized_ids = tokenizer(
  input_words,
  truncation=True, # 是否截斷序列
  # is_split_into_words: Whether or not the input is already pre-tokenized (e.g., split into words).
  # If set to True, the tokenizer assumes the input is already split into words (for instance, by splitting it on whitespace) which it will tokenize.
  # This is useful for NER or token classification.
  is_split_into_words=True)
pprint(input_tokenized_ids, compact=True)

經過分詞器後將得到一個包含 `attention_mask` 及 `input_ids` 的字典，其中 `attention_mask` 用於指示哪些 tokens 是模型需要關注，哪些是 padding tokens 可以忽略，`input_ids` 則是 tokens 的索引或對應到辭典 (vocab) 的 ID。

#### 還原編碼後的文字

我們亦可以透過 `tokens()` 方法將 `input_ids` 轉換回 tokens。

分詞器添加了模型使用的特殊標記（[CLS] 在開頭和 [SEP] 在結尾），並且大多數單詞保持不變。然而，某些單詞 (word) 會被分為數個子詞 (subword)。

In [32]:
# 如我們所見，分詞器添加了模型使用的特殊標記（[CLS] 在開頭和 [SEP] 在結尾），
# 並且大多數單詞保持不變。然而，某些單詞 (word) 會被分為數個子詞 (subword)，如: Bar, ##eil 和 ##ly
pprint(input_tokenized_ids.tokens(), compact=True)

於是，經過分詞器後，這導致了我們的輸入和標籤之間的不匹配，從兩者長度便可窺知一二。

In [33]:
# 原始資料
print(f'length of input_words: {len(input_words)}')
print(f'length of input_labels: {len(input_labels)}')
# 這導致了我們的輸入和標籤之間的不匹配
print(f'length of token id: {len(input_tokenized_ids.tokens())}')

#### 還原編碼後的位置

感謝 fast tokenizer 我們可以輕鬆地將每個 Token 藉由 `word_ids()` 映射到其對應的單詞位置或 Word IDs。

舉例：

| Word    | Token | Word ID |
|---------|-------|---------|
|         | [CLS] | None    |
| My      | My    | 0       |
| name    | name  | 1       |
| is      | is    | 2       |
| Ludmila | Lu    | 3       |
|         | ##d   | 3       |
|         | ##mi  | 3       |
|         | ##la  | 3       |
|         | [SEP] | None    |


In [34]:
# `word_ids` return the list of tokens (sub-parts of the input strings after word/subword splitting and before conversion to integer indices)
# at a given batch index (only works for the output of a fast tokenizer).
pprint(input_tokenized_ids.word_ids(), compact=True)


[CLS] 和 [SEP] 並不對應到任何單詞，所以它們的 Word ID 為 None。其他則是以 0 開始的 Word ID，如遇到子詞 (subword) 則會共用相同的 Word ID。

根據上述的了解，我們依序將輸入，編碼後的文字，以及編碼後的位置對比，幫助我們理解分詞器的行為。

In [35]:
# 資料對比
print('=== Tokenizer 前 (Words 及 Labels) ===')
display_words_labels(dataset['train'][0], max_display)

print()
print('=== Tokenizer 後 (還原編碼後的文字) ===')
pprint(input_tokenized_ids.tokens()[:max_display], compact=True)

print()
print('=== Tokenizer 後 (還原編碼後的位置) ===')
pprint(input_tokenized_ids.word_ids()[:max_display], compact=True)

### 資料預處理

#### 重新校準標籤

我們可以搭配 `word_id` 擴展標籤 (Label) 以匹配單詞 Token ID，將標籤與分詞後的標記對齊。

1. 首先，我們將應用的規則是特殊 Token 獲得 -100 標籤。這是因為默認情況下，-100 是我們在損失函數中被忽略的索引。
2. 然後，每個相同 Word ID 的 Token 獲得與其所在單詞相同的標籤，因為它們是同一實體的一部分。

定義一個函數接受兩個參數：`word_ids` 和 `labels`，並返回重新校準後的標籤。

In [36]:
def align_labels_with_tokens(
        word_ids: list,
        labels: list,) -> list:
    new_labels = [] # 用於存儲對齊後的新標籤
    current_word_id = None # 用於追踪當前的單詞 ID
    for word_id in word_ids:
        if word_id is None: # None 代表特殊 Token
            label = -100 # 特殊 Token 獲得 -100 的標籤
        elif word_id != current_word_id: # Word ID 改變，代表新的 Token
            current_word_id = word_id # 更新 Word ID
            label = -100 if word_id is None else config.tag2id.get(labels[word_id]) # 取得對應的標籤
        else: # 與前一個 Token 相同的字
            label = config.tag2id.get(labels[word_id])
        new_labels.append(label) # 附加新的標籤
    # 返回新的標籤
    return new_labels

再來檢視一次原先 `input_labels` 中的標籤。

In [37]:
# 先將其轉換為標籤編碼
input_label_ids = [config.tag2id.get(label) for label in input_labels]
pprint(input_label_ids, compact=True)

經過重新校準後，我們可以看到標籤已經與分詞後的標記對齊，且長度相同。

In [38]:
input_label_ids_aligned = align_labels_with_tokens(
  input_tokenized_ids.word_ids(), # 透過 word_ids 取得對應的 word id
  input_labels, # 原始標籤
)
pprint(input_label_ids_aligned, compact=True)

In [39]:
# 顯示對齊後的標籤，兩者數量應該相同
print(f'length of token id: {len(input_tokenized_ids.tokens())}')
print(f'length of labels: {len(input_label_ids_aligned)}')

經過重新校準後，我們的標籤與 Token 一一對應。

| Word    | Token | Word ID | Label          |
|---------|-------|---------|----------------|
|         | [CLS] | None    | -              |
| My      | My    | 0       | O              |
| name    | name  | 1       | O              |
| is      | is    | 2       | O              |
| Ludmila | Lu    | 3       | B-NAME_STUDENT |
|         | ##d   | 3       | B-NAME_STUDENT |
|         | ##mi  | 3       | B-NAME_STUDENT |
|         | ##la  | 3       | B-NAME_STUDENT |
|         | [SEP] | None    | -              |


In [40]:
def display_tokens_labels(tokens, label_ids):
    line1 = ""
    line2 = ""
    for token, label_id in zip(tokens, label_ids): # 逐一取出 token 與 label_id
        # 將整數標籤轉換為 BIO 標籤方便比較
        label = config.id2tag.get(label_id, '-')
        # 計算 token 與 label 的最大長度, 並將 token 與 label 用空白補齊至相同長度
        max_length = max(len(token), len(label))
        line1 += token + " " * (max_length - len(token) + 1)
        line2 += label + " " * (max_length - len(label) + 1)
    pprint(line1, width=200)
    pprint(line2, width=200)

# 顯示對齊後的 tokens 與 labels
display_tokens_labels(
    input_tokenized_ids.tokens()[:max_display],
    input_label_ids_aligned[:max_display])

#### 定義預處理函數

要預處理整個數據集，我們需要對所有輸入進行分詞，並對所有標籤應用 `align_labels_with_tokens()`。為了利用快速分詞器的速度，最好一次分詞大量文本，因此我們將編寫一個批次處理函數。

In [41]:
def preprocess_function(dataset):
    # 使用分詞器對資料集進行分詞
    tokenized_inputs = tokenizer(
        dataset['words'],
        truncation=True,
        is_split_into_words=True
    )
    labels = dataset['labels']
    new_labels = []
    for i, labels in enumerate(labels):
        # 取得對應的 word_ids
        word_ids = tokenized_inputs.word_ids(i)
        # 對齊標籤
        new_labels.append(
            align_labels_with_tokens(word_ids, labels)
        )
    # 更新 labels 欄位，請留意這邊的 labels 欄位是整數標籤，而非 BIO 標籤
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [42]:
input_tokenized_ids = preprocess_function(dataset['train'][:1])

In [43]:
pprint(input_tokenized_ids, compact=True)

經過 `preprocess_function` 後將得到一組先前我們使用分詞器的 `attention_mask` 及 `input_ids`，也會增加一個對齊後的 `labels`。

In [44]:
# 顯示對齊後的 tokens 與 labels
display_tokens_labels(
    input_tokenized_ids.tokens()[:max_display],
    input_tokenized_ids['labels'][0][:max_display])

請留意經過資料預處理後，`labels` 欄位是整數標籤，而不是 BIO 字串標籤。

#### 批次處理資料

使用 `Dataset.map()` 方法，選項設置為 `batched=True`。

In [45]:
tokenized_dataset = dataset.map(
    preprocess_function, # 對資料集進行分詞與標籤對齊
    batched=True, # 是否以批次進行處理
    remove_columns=dataset['train'].column_names, # 移除不必要的欄位
)

In [46]:
# 顯示前 first_n_data 筆資料
pd.DataFrame(tokenized_dataset['train'].select(range(first_n_data)))

### 資料校對器 (Data Collator)

在微調語言模型時，使用 data collator 是為了有效地準備和處理批次數據。以下是使用 data collator 的幾個主要原因：

* 動態填充 (Dynamic Padding): 不同長度的序列需要填充到相同的長度，以便能夠在同一批次中進行處理。Data collator 可以自動計算每個批次的最大長度，並對序列進行適當的填充。

* 批次處理 (Batch Processing): Data collator 可以將多個樣本組合成一個批次，這樣可以更高效地利用計算資源，特別是在使用 GPU 或 TPU 時。

* 生成注意力掩碼 (Attention Masks): 在填充序列時，data collator 會生成相應的注意力掩碼 (attention masks)，以確保模型只關注實際的數據部分，而忽略填充部分。

* 簡化代碼 (Code Simplification): 使用 data collator 可以簡化數據處理的代碼，減少手動處理數據的繁瑣步驟，讓開發者專注於模型設計和訓練。

總之，data collator 在微調語言模型時提供了便利和效率，確保數據能夠以一致且高效的方式進行處理。

這個課程中，我們不能僅使用 `DataCollatorWithPadding`，因為它只填充輸入（Input IDs, Attention Mask）。在這裡，我們的標籤 (Label) 應該以與輸入完全相同的方式進行填充，以保持相同的大小，使用 -100 作為填充值，以便在損失計算中忽略相應的預測。

於是我們可以利用 `DataCollatorForTokenClassification` 完成。與 `DataCollatorWithPadding` 一樣，它需要使用預處理輸入的分詞器：

>
> * [DataCollatorWithPadding](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorWithPadding) that will dynamically pad the inputs received.
> * [DataCollatorForTokenClassification](https://huggingface.co/docs/transformers/main_classes/data_collator#transformers.DataCollatorForTokenClassification) that will dynamically pad the inputs received, as well as the `labels`.
>

In [47]:
data_collator = DataCollatorForTokenClassification(
  tokenizer=tokenizer
)

In [48]:
# 展示 DataCollatorForTokenClassification 的輸出, 標籤以 -100 表示 padding
features = [tokenized_dataset['train'][i] for i in range(first_n_data)]
batch = data_collator(features)

In [49]:
pprint(batch)

這裡要注意的主要是第一個例子與第二個例子長度不一，所以長度不足的例子的 `input_ids` 和 `attention_mask` 已經在右側填充了一個 [PAD] 標記（其 ID 是 0）。

In [50]:
pprint(batch["labels"])

類似地，我們可以看到 `labels` 已用 -100 填充，以確保填充標記被損失函數忽略。

我們終於擁有了訓練所需的所有的前期準備！我們現在只需要使用標準參數實例化訓練器。

### 模型評估函數

在訓練過程中包含度量標準通常有助於評估模型的性能。您可以使用 Evaluate 庫快速加載評估方法。對於這個任務，請加載 [seqeval](https://huggingface.co/docs/evaluate/a_quick_tour) 框架。Seqeval 實際上會生成多個分數：precision, recall, F1, 和 accuracy。

* Precision: 精確率，是指所有被標記為正的樣本中實際為正的比例。

$\ Precision = \frac{\text{correctly classified actual positives}}{\text{everything classified as positives}} = \frac{TP}{TP + FP} $

* Recall: 召回率，是指所有實際為正的樣本中被標記為正的比例。

$\ Recall = \frac{\text{correctly classified actual positives}}{\text{all actual positives}} = \frac{TP}{TP + FN} $

* F1: F1 值是精確率和召回率的調和平均值，用於綜合考慮精確率和召回率。

$\ F1 = 2 \times \frac{Precision \times Recall}{Precision + Recall} $

* Accuracy: 準確率，是指所有被正確分類的樣本數量與總樣本數量之比。

$\ Accuracy = \frac{\text{correctly classifications}}{\text{total classifications}} = \frac{TP + TN}{TP + TN + FP + FN} $


In [51]:
seqeval = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    # Unpack logits and labels from the input
    logits, labels = eval_preds

    # Convert logits to the index of the maximum logit value
    predictions = np.argmax(logits, axis=-1)

    # Map predictions and labels to their corresponding label names, ignoring padding (-100)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    # Compute evaluation metrics using seqeval
    results = seqeval.compute(predictions=true_predictions, references=true_labels)

    # Return the computed metrics
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### 載入預訓練模型

您現在可以開始訓練您的模型了！使用 `AutoModelForTokenClassification` 加載預訓練的模型，並指定預期標籤的數量和標籤映射：

需要留意，因為預訓練模型的分類數為 9，所以我們需要重新設定模型的分類數，且忽略預訓練模型的分類層。

* `num_labels`: 指定了任務中標籤的數量。這告訴模型有多少個不同的標籤需要進行分類。

* `ignore_mismatched_sizes`: 表示在加載模型時忽略預訓練模型的標籤數量與我們的標籤數量不匹配的情況。這在我們的任務標籤數量與預訓練模型的標籤數量不同時非常有用。

* `id2label`: 是一個字典，用於將整數標籤映射到 BIO 標籤。這有助於在模型輸出時將整數標籤轉換為可讀的 BIO 標籤。

* `label2id`: 是 `id2label` 的反向映射，用於將 BIO 標籤映射回整數標籤。

以下程式碼從預訓練模型中加載一個標記分類模型，並根據特定任務的需求進行配置。它指定了模型名稱、任務標籤數量，並設置了忽略不匹配大小的選項。此外，它還提供了整數標籤與 BIO 標籤之間的映射，便於模型在輸入和輸出時進行轉換。這樣可以確保模型能夠正確處理特定任務中的標籤。


In [52]:
model = AutoModelForTokenClassification.from_pretrained(
  config.model_name,
  num_labels=config.num_tags, # 任務標籤數量
  ignore_mismatched_sizes=True, # 忽略不匹配的大小，預訓練模型的標籤數量與我們的標籤數量不同
  id2label=config.id2tag, # 整數標籤到 BIO 標籤的映射
  label2id=config.tag2id, # BIO 標籤到整數標籤的映射
)

In [53]:
# 查看可訓練的參數量約 65M
print('Parameters: {:,}, Trainable Parameters: {:,}'.format(
  model.num_parameters(),
  model.num_parameters(only_trainable=True)))

### 訓練參數設定

用於設定訓練過程中的各種參數，如學習率、批次大小、梯度累積步數、訓練 epoch 數、權重衰減等。

* `output_dir` 指定了訓練輸出的目錄。
* `eval_strategy` 和 `save_strategy` 設定為 'epoch'，表示每個 epoch 都會進行評估和儲存。
* `load_best_model_at_end` 設定為 `True`，表示訓練結束後會載入最佳模型。
* `report_to` 設定為 'none'，禁用了 wandb 報告。
* `save_total_limit` 設定了最多儲存 5 個 checkpoints。

In [ ]:
training_args = TrainingArguments(
  output_dir='sample_data/train_output_pii_detection',
  learning_rate=config.lr,
  per_device_train_batch_size=config.train_batch_size,
  per_device_eval_batch_size=config.eval_batch_size,
  num_train_epochs=config.epochs,
  weight_decay=config.weight_decay,
  eval_strategy='epoch', # 每個 epoch 評估一次
  save_strategy='epoch', # 每個 epoch 儲存一次
  load_best_model_at_end=True,
  report_to='none', # Disable wandb on colab
  save_total_limit=5, # 最多儲存 5 個 checkpoints
)

### 訓練器初始化

用於初始化訓練器，並開始訓練模型。

* `model` 是要訓練的模型。
* `tokenizer` 是用於處理文本的分詞器。
* `train_dataset` 和 `eval_dataset` 是訓練和評估數據集。
* `data_collator` 是用於整理數據的數據整理器。
* `compute_metrics` 是用於計算度量標準的函數。

In [54]:
trainer = Trainer(
  model=model,
  tokenizer=tokenizer,
  args=training_args,
  train_dataset=tokenized_dataset['train'],
  eval_dataset=tokenized_dataset['validation'],
  data_collator=data_collator,
  compute_metrics=compute_metrics,
)

### 開始訓練

In [55]:
# 開始訓練，這可能需要一些時間
trainer.train()

訓練完成後，您可以通過運行 `Trainer.evaluate()` 方法在驗證集上評估模型的性能。它會計算模型的損失和其他評估指標，並返回這些結果。這對於了解模型在未見數據上的表現非常有用。

In [56]:
trainer.evaluate()

#### 保存微調模型

In [57]:
# 儲存模型
trainer.save_model(config.saved_model_path)

### 釋放資源

In [ ]:
# import garbage collector
import gc

# 釋放 GPU 記憶體
del trainer
del tokenizer

model.to('cpu')
del model

torch.cuda.empty_cache()

gc.collect()

## 評估微調模型

### 載入微調分詞器 (Tokenizer)

從已經完成訓練的模型取得 Tokenizer。

In [58]:
tokenizer = AutoTokenizer.from_pretrained(
  config.saved_model_path,
)

### 載入微調後模型

In [59]:
ft_model = AutoModelForTokenClassification.from_pretrained(
  config.saved_model_path,
).to(device)

請留意分類數已經從預訓練模型的 9 類，變成 10 類。

In [ ]:
pprint(ft_model)

### Fine-tuning 後的表現

In [ ]:
# 載入微調模型，宣告分類器
classifier = pipeline(
  task="token-classification",
  model=ft_model,
  tokenizer=tokenizer,
  device=device,
)

In [ ]:
# 顯示微調模型預測結果，僅顯示有被標注的部分
for val in dataset['test']: # 逐一取出測試資料
  print(f'輸入: {val["text"]}')
  display_words_entity(val["text"], classifier)
  print()